In [98]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")   # se notebook sta in notebooks/ e src/ è a fianco

import src.data as dt
import src.models as md
import src.features as ft
import src.predict as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [99]:
(X_train, X_val, y_train, y_val), X, y = dt.load_train_set()

## 2. Feature Selection

In [63]:
importance_dfs = ft.get_feature_importances(md.MODELS, X_train, y_train)
importance_df = ft.combine_feature_importances(importance_dfs)

print(importance_df.shape)
print(len(importance_df))
importance_df.head()

Values to compute: .
(66, 7)
66


,feature,importance_xgb,importance_ctb,importance_lgb,importance_svr,importance_mlp,importance_knn
33,1stFlrSF,0.050795,0.055531,0.047587,0.077864,0.126215,0.032349
34,2ndFlrSF,0.014597,0.026175,0.018163,0.062962,0.112680,0.018871
59,3SsnPorch,0.000000,0.000007,0.000000,0.000000,0.000000,0.000000
41,BedroomAbvGr,0.002850,0.008655,0.001769,0.004588,0.013974,0.008968
7,BldgType,0.003606,0.003777,0.000374,0.000508,0.009924,0.004133


In [64]:
comparison_df = ft.get_feature_comparison(importance_df)

ft.plot_importance_comparison(comparison_df)
ft.plot_rank_comparison(comparison_df)

In [ ]:
feature_numbers = [10, 15, 20, 25, 30, 35]
scores = ft.evaluate_feature_number_per_models(md.MODELS, feature_numbers, X_train, y_train, features_by_importance)

In [72]:
ft.plot_model_scores(scores, feature_numbers, 3)


## 3. Model optimization

In [180]:
params = {}
cv_scores = {}
val_scores = {}
studies = {}

features_by_importance = ft.get_features_by_importance(comparison_df)

for key in ['svr']:
    print(f'\nEvaulating: {md.MODELS[key]["name"]}')
    params[key], cv_scores[key], val_scores[key], studies[key] = md.evaluate_best_feature_number(
        md.MODELS[key],
        features_by_importance,
        [10, 20, 30, 40, 50, 60, 66],
        X_train,
        y_train,
        X_val,
        y_val,
    )


Evaulating: Support Vector Regressor
processsing number 10
cv_score: 0.29541803545501105, val_score: 0.3686875570912747, param: {'C': 4.953085181471394, 'gamma': 0.03729820004280494, 'epsilon': 0.19436895050228192}
processsing number 20
cv_score: 0.2757932925190878, val_score: 0.33696576384762117, param: {'C': 6.794678001616118, 'gamma': 0.010374764604523221, 'epsilon': 0.06730973266680537}
processsing number 30
cv_score: 0.26748685505163783, val_score: 0.32916297849566967, param: {'C': 43.41979693532774, 'gamma': 0.0017035916230312836, 'epsilon': 0.1189828759688514}
processsing number 40
cv_score: 0.26765326028873, val_score: 0.3204915505941722, param: {'C': 26.23940627507973, 'gamma': 0.0011822618005954337, 'epsilon': 0.09702638672304388}
processsing number 50
cv_score: 0.266613121941279, val_score: 0.32357635295207826, param: {'C': 22.499228441928718, 'gamma': 0.0012188689291704408, 'epsilon': 0.039826029621171384}
processsing number 60
cv_score: 0.2668612004204606, val_score: 0.31

In [184]:
md.plot_model_scores({'svr': cv_scores['svr']}, {'svr': val_scores['svr']}, [10, 20, 30, 40, 50, 60, 66])

NameError: name 'sp' is not defined

In [ ]:
print(val_scores['svr'])
print(cv_scores['svr'].head())

{'10': 0.3686875570912747, '20': 0.33696576384762117, '30': 0.32916297849566967, '40': 0.3204915505941722, '50': 0.32357635295207826, '60': 0.313037669291896, '66': 0.3188453664289792}
         10        20        30        40        50        60        66
0  0.284404  0.267633  0.262768  0.266679  0.270730  0.268948  0.275522
1  0.301637  0.290450  0.284690  0.273468  0.269805  0.270229  0.276617
2  0.287564  0.239473  0.237204  0.238795  0.238407  0.231917  0.236631
3  0.301856  0.284561  0.275272  0.271173  0.263935  0.270468  0.272790
4  0.301631  0.296850  0.277500  0.288151  0.290189  0.292744  0.297605


## 4. Predictions

In [ ]:
n = 30

In [134]:
X_test = dt.load_test_data()
raw_predictions = pd.get_raw_predictions(md.MLP, params_mlp[str(30)], X[features_by_importance[:n]], y, X_test[features_by_importance[:n]])
dt.prepare_submission(raw_predictions)